<a href="https://colab.research.google.com/github/andresgabas/DS-Coderhouse/blob/main/ProyectoDS_ParteI_Gab%C3%A1s.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mlxtend --upgrade --no-deps

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.plotting import plot_decision_regions
import warnings
warnings.filterwarnings("ignore")

Abstract:

En este proyecto se trabajará con un dataset obtenido del sitio web www.kaggle.com.

El dataset elegido es un archivo .csv que contiene información de pacientes que sufrieron o no infartos (no se aclara la fuente)
El infarto es la segunda causa de muerte más frecuente en el mundo, por lo tanto la posibilidad de crear un modelo que permita la prediccón de un infarto podría prevenir a las personas con riesgo de tomar comsciencia sobre su salud.
Se hará un análisis utilizando diferentes librerías de Python, para conocer la base de datos y ver que variables se podrán utilizar para elaborar un modelo que permita predecir si una persona puede llegar a tener un infarto.

Variables incorporadas en el dataset (Entre paréntesis figuran los nombres de las columnas que se utilizarán en el anásisis para mayor comodidad):
*   ID (ID): columna con un número que identifica a cada paciente. Se eliminará ya que no se va a utilizar.
*   Gender (genero): Género, masculino o femenino. el género otro se eliminó por ser poco representativo.
*   Age (edad): Edad de los participantes del estudio. Se considerarán los mayores de 18.
*   Hypertension (hipertension): Hipertenso si(1) o no (0).
*   Heart Disease (enfermedad_corazon): Enfermedad cardíaca si(1) o no (0).
*   Ever married (Casado): Si o no
*   work_type (tipo_trabajo): Tipo de trabajo private, self employed, children, govt job or never worked.
*   Residence_type(tipo_residencia): Urbana o rural.
*   avg_glucose_level(glucosa_promedio): Nivel de glucosa promedio.
*   BMI(IMC): indice de masa muscular. En este caso se eliminarán los valores NaN.
*   smoking_status(fumador): Nunca fumó, desconocido, ex fumador, fumador.
*   stroke(infarto): Tuvo infarto si(1) o no (0).

Se buscará comprobar las siguientes hipótesis
*   Si trabajar con un dataset desequilibrado, puede afectar a la predictabilidad del modelo y conducir a errores en las predicciones.
*   Se buscará comprobar si las personas que tienen enfermedad cardíaca e hipertensión son más propensas a tener infartos.
*   Se buscará averiguar si altos valores de IMM y glucosa promedio son factores que conducen a una probabilidad más alta de tener infartos.
*   Se buscará ver si según el tipo de trabajo, hay más posibilidades de tener infartos.
*   Se buscará comprobar si según se viven en zonas urbanas o rurales, hay más posibilidades de tener infartos.     


In [ ]:
url = 'https://raw.githubusercontent.com/andresgabas/DS-Coderhouse/main/healthcare-dataset-stroke-data.csv'
Stroke = pd.read_csv(url)


In [ ]:
Stroke.drop('id', axis=1)

In [ ]:
Stroke.columns = ['Id','genero','edad','hipertension','enfermedad_corazon','casado','tipo_trabajo','tipo_residencia','glucosa_promedio','IMC','fumador','infarto']

De la columna género, se eliminará la fila correspondiente al género "other" porque al ser un único caso , no es representativo.

In [ ]:
Stroke = Stroke[Stroke['genero'] != 'Other']

En este paso se eliminarán los valores NaN ya que no aportan al dataset y no podrían ser reemplazados por ceros porque el valor 0 en el 'indice de masa muscular no tiene sentido.

In [ ]:
Stroke = Stroke.dropna(subset=['IMC'])

In [ ]:
Stroke = Stroke[Stroke['IMC'] <= 50]

In [ ]:
Stroke = Stroke[Stroke['edad'] >= 18]

In [ ]:
Stroke.count()

In [ ]:
stroke_counts = Stroke['infarto'].value_counts()
plt.pie(stroke_counts, labels=['Sanos', 'Con infarto'], autopct='%1.1f%%')
plt.title('Porcentaje de pacientes con infartos')
plt.show()


Para comenzar se analizó el porcentaje de pacientes que sufrieron un infarto. Se observa en el piechart que solo el 5.2% de los pacientes sufrieron un infarto. Este gráfico nos muestra que el dataset está muy desbalanceado, esto puede generar problemas en la eficacia de nuestro algoritmo.  

In [ ]:
gender_counts = Stroke['genero'].value_counts()
ax = gender_counts.plot(kind='bar', color=['pink','blue'])
ax.set_title('Distribución de pacientes por género')
ax.set_ylabel('Cantidad')
ax.set_xlabel('Género')

En este gráfico se observa que la cantidad de participantes mujeres es mayor que la hombres.

In [ ]:
plt.hist(Stroke['edad'], edgecolor='black',rwidth=0.8)
plt.xlabel('Edad')
plt.ylabel('Frecuencia')
plt.title('Distribución de edades')
plt.show()

En este histograma de edades de pacientes se observa que la distribución de la edad los pacientes no es normal. Además podemos observar que la mayoría de los pacientes tienen entre 40 y 60 años, más cerca de los sesenta.


In [ ]:
plt.hist(Stroke['IMC'], bins=20, edgecolor='black')
plt.xlabel('IMC')
plt.ylabel('Frecuencia')
plt.title('Distribución de indice de masa corporal')
plt.show()

A partir del gráfico de frecuencias del indice de masa corporal observamos una distribución de las muestras corridas hacia la izquierda.
Al principio del análsis se mencionó que es dataset está muy desequilibrado, por lo tanto se tomaran los valores de indice masa corporal mayor a 60 como outliers y no tendrán en cuenta en este análisis.

In [ ]:
sns.countplot(x='fumador', data=Stroke, hue = 'infarto')
plt.title('Distribución de pacientes por estado de fumador')
plt.show()

En este gráfico gracias al atributo hue que permite agregar una variable más a los ya tenidas en cuenta en el gráfico, podemos decir con seguridad que la mayor parte de los pacientes nunca fumó, también hay una gran porporción de pacientes que se desconoce si fumaron o fuman en la actualidad.


In [ ]:
sns.countplot(x='hipertension', data=Stroke, hue = 'infarto')
plt.title('Distribución de pacientes por hipertensión')
plt.show()

In [ ]:
sns.countplot(x='enfermedad_corazon', data=Stroke, hue = 'infarto')
plt.title('Distribución de pacientes por enfermedad del corazon')
plt.show()

In [ ]:
sns.countplot(x='tipo_residencia', data=Stroke, hue = 'infarto')
plt.title('Distribución de pacientes por tipo de residencia')
plt.xlabel('Tipo de Residencia')
plt.ylabel('Cantidad de Pacientes')
plt.show()

In [ ]:
sns.countplot(x='casado', data=Stroke, hue = 'infarto')
plt.title('Distribución de pacientes por estado civil')
plt.xlabel('Casados')
plt.ylabel('Cantidad de Pacientes')
plt.show()

In [ ]:
sns.countplot(x='tipo_trabajo', data=Stroke, hue = 'infarto')
plt.title('Distribución de pacientes segun si trabajan')
plt.xlabel('Si trabajan y en que sector')
plt.ylabel('Cantidad de Pacientes')
plt.show()

In [ ]:
sns.scatterplot(x = 'edad', y = 'glucosa_promedio', data = Stroke, hue = 'infarto')

Se realizó este gráfico de dispersión con el objetivo de ver si había una correlación entre la edad y la glucosa promedio, el gráfico no nos aporta ninguna correlación entre estos factores, pero si se observa que la mayoría de los pacientes con infarto, se corresponden con las zonas de alta glucosa promedio y mayor edad.

In [ ]:
sns.scatterplot(x = 'edad', y = 'IMC', data = Stroke, hue = 'infarto')

En este segundo gráfico de dispersión se trató de ver si existe una correlación entre la edad y el índice de masa muscular pero dada la distribución de los puntos, no se observa una correlación positiva o negativa, sino mas bien una correlación neutra.

In [ ]:
sns.scatterplot(x = 'glucosa_promedio', y = 'IMC', data = Stroke, hue = 'infarto')

En este nuevo Scatterplot se trató de ver si hay una correlación positiva o negativa entre la glucosa promedio y el índice de masa muscular.

El gráfico nos permite decir que no hay una correlación positiva o negativa.

Luego del análisis exploratorio de datos, se procederá a utilizar distintos algoritmos de clasificación de machine learning de la librería Scikit Learn para evaluar las distintas hipótesis propuestas.

In [ ]:
# prompt: Crear variable x con imm y glucosa promedio

X = Stroke[['glucosa_promedio','edad']]
y = Stroke['infarto']

In [ ]:
#Convert x y to numpy arrays
X = np.array(X)
y = np.array(y)

In [ ]:
# Paso 1
# Importar las librerias del modelo
from sklearn.linear_model import LogisticRegression

# Paso 2
# Crear la instancia del modelo
clf = LogisticRegression()

# Paso 3
# Entrenar el modelo con los datos y la etiqueta
clf.fit(X, y)

In [ ]:
plt.figure(figsize=(12, 12))
ax=plot_decision_regions(X, y, clf=clf)
ax.legend(['No infarrtado','infarto'])
plt.xlabel('Glucosa promedio' )
plt.ylabel('Edad')
plt.title('Comparación infartados no infartados según edad y glucosa promedio')
plt.show()

In [ ]:
# Paso 1
# Importar las librerias del modelo
from sklearn.tree import DecisionTreeClassifier # Import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder # Import LabelEncoder for encoding categorical features

# Paso 2
# Crear la instancia del modelo
clf = DecisionTreeClassifier(max_depth=4)

# Paso 3
# Entrenar el modelo con los datos y la etiqueta
X= Stroke[['IMC', 'genero']] # Select the 'genero' column directly
y= Stroke['infarto']

# Encode the 'genero' column to numerical values
le = LabelEncoder()
X['genero'] = le.fit_transform(X['genero']) # Convert 'genero' to numerical labels

clf = clf.fit(X, y) # Now fit the model with numerical data

In [ ]:
import graphviz
from sklearn import tree

# Convert class names to strings
class_names = [str(name) for name in y.unique()]  # Convert each class name to string

dot_data = tree.export_graphviz(clf, out_file=None,
                                feature_names=X.columns,
                                class_names=class_names,  # Use the converted class names
                                filled=True, rounded=True,
                                special_characters=True)
graph = graphviz.Source(dot_data)
graph